# Create PNG image files for 30 second clips.


Source: fma_small mp3 file.

Target: smMELsg_30sec_Train_Test.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import utils

SG_DIR =     './fma_small_specgram'
SG_IMG_DIR = './fma_small_specgram_img'

In [ ]:
import multiprocessing
import os

cores = multiprocessing.cpu_count()
print(f'Cores: {cores}')
print(f'OS CPU count: {os.cpu_count()}')
print(f'Affinity: {os.sched_getaffinity(0)}')

Cores: 8
OS CPU count: 8
Affinity: {0, 1, 2, 3, 4, 5, 6, 7}


## Load meta-data.

In [ ]:
tracks = utils.load('data/fma_metadata/tracks.csv')
subset = tracks.index[tracks['set', 'subset'] <= 'small']
assert subset.isin(tracks.index).all()
tracks = tracks.loc[subset]
labels = tracks['track', 'genre_top']
tracks.shape

(8000, 52)

In [ ]:
train = tracks.index[tracks['set', 'split'] == 'training']
train_lables = labels[train]
val = tracks.index[tracks['set', 'split'] == 'validation']
val_lables = labels[val]
test = tracks.index[tracks['set', 'split'] == 'test']
test_lables = labels[test]
print('{} training examples, {} validation examples, {} testing examples'.format(*map(len, [train, val, test])))

6400 training examples, 800 validation examples, 800 testing examples


In [ ]:
genres = ['Hip-Hop', 'Pop', 'Folk', 'Experimental', 'Rock',
          'International', 'Electronic', 'Instrumental']

## Create target directories.

In [ ]:
import os

os.makedirs(SG_IMG_DIR, exist_ok=True)
os.makedirs(SG_IMG_DIR + '/train', exist_ok=True)
os.makedirs(SG_IMG_DIR + '/val', exist_ok=True)
os.makedirs(SG_IMG_DIR + '/test', exist_ok=True)
for g in genres:
  os.makedirs(SG_IMG_DIR + '/train/' + g, exist_ok=True)
  os.makedirs(SG_IMG_DIR + '/val/' + g, exist_ok=True)
  os.makedirs(SG_IMG_DIR + '/test/' + g, exist_ok=True)

In [ ]:
'filename.png'[:-3]

'filename.'

## Using MP3, create mel-spectrograma and then create PNG image file.

### Create paths for source MP3 and target image files.

In [ ]:
import os

def load_mel_sg(track):
  arr = np.load(track[2])
  return arr

def load_split_genre(_split, _genre):
  genre_sg_lst = []
  MAX_SHAPE = 0
  dir_path = f'{SG_DIR}/{_split}/{_genre}'
  for filename in os.listdir(dir_path):
    track_path = f'{dir_path}/{filename}'
    track_image_path = f'{SG_IMG_DIR}/{_split}/{_genre}/{filename[:-3]}png'
    sg = np.load(track_path)
    if sg.shape[0] == 128 and sg.shape[1] >= 1290:  # IGNORING SPECTROGRAMS WITH LESS THAN 1290 FEATURES.
      sg = sg[:,:1290]
      genre_sg_lst.append( (track_image_path, sg) )
  return genre_sg_lst

def load_split(_split):
  split_data = {}
  for genre in genres:
    genre_data = load_split_genre(_split, genre)
    split_data[genre] = genre_data
  return split_data

train_data = load_split('train')
val_data = load_split('val')
test_data = load_split('test')

In [ ]:
for genre in genres:
  print(f'{genre}: {len(train_data[genre])}')

Hip-Hop: 797
Pop: 800
Folk: 800
Experimental: 799
Rock: 799
International: 800
Electronic: 799
Instrumental: 800


In [ ]:
print(train_data['Pop'][0][0])
train_data['Pop'][0][1].shape

./fma_small_specgram_img/train/Pop/113933.png


(128, 1290)

## Create image files.

In [ ]:
%%time
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from multiprocessing import Pool
import functools
from functools import partial

def create_mel_sg_img(track):
  try:
    if os.path.isfile(track[0]): return
    fig = plt.Figure(frameon=False)
    canvas = FigureCanvas(fig)
    plt.imshow(track[1], origin='lower', aspect='auto')
    plt.grid(False)
    plt.axis('off')
    plt.gca().get_xaxis().set_visible(False)
    plt.gca().get_yaxis().set_visible(False)
    plt.tight_layout()
    plt.savefig(track[0], bbox_inches='tight', pad_inches=0)
  except Exception as e:
    print(f'Failed to creatae imagae for {track[0]}')
    print(e)

def create_split_data(_track_list):

  mel_sg_shape_map = {}

  for idx, track in enumerate(_track_list):
    try:
      if os.path.isfile(track[0]): continue
      create_mel_sg_img(track)
    except Exception as e:
      print(f'Failed creating file {track[0]}')
      print( e)
      continue
    if idx % 500 == 0: print(f'Count: {idx}')

#train_data['Pop'][0][0]
plt.ioff()

#create_split_data(train_data['Pop'])

pool = Pool(processes=7)
pool.map(create_mel_sg_img, train_data['Pop'])
pool.close()
pool.join()

r = plt.ion()

CPU times: user 22.1 s, sys: 3.39 s, total: 25.5 s
Wall time: 1h 12min 18s


In [ ]:
plt.ioff()

pool = Pool(processes=7)
pool.map(create_mel_sg_img, train_data['Rock'])
pool.close()
pool.join()

r = plt.ion()

In [ ]:
plt.ioff()

pool = Pool(processes=7)
pool.map(create_mel_sg_img, train_data['Electronic'])
pool.close()
pool.join()

r = plt.ion()

## Create test dataset.

In [ ]:
plt.ioff()
for genre in ['Pop', 'Rock', 'Electronic']:
  pool = Pool(processes=7)
  pool.map(create_mel_sg_img, test_data[genre])
  pool.close()
  pool.join()
r = plt.ion()

## Create validation dataset.

In [ ]:
plt.ioff()
for genre in ['Pop', 'Rock', 'Electronic']:
  pool = Pool(processes=7)
  pool.map(create_mel_sg_img, val_data[genre])
  pool.close()
  pool.join()
r = plt.ion()

In [ ]:
import cv2

img_shapes = {}
for idx, genre in enumerate(['Rock','Electronic','Pop']):
  img_dir_path = f'{SG_IMG_DIR}/train/{genre}'
  for filename in os.listdir(img_dir_path):
    img_path = f'{img_dir_path}/{filename}'
    im = cv2.imread(img_path)
    try:
      if im.shape not in img_shapes:
        img_shapes[im.shape] = 1
      else:
        img_shapes[im.shape] += 1
    except Exception as e:
      print(img_path)
      continue

img_shapes

{(450, 610, 3): 2398}

## Check dimensions of image files.

In [ ]:
import cv2

img_shapes = {}
for idx, genre in enumerate(['Rock','Electronic','Pop']):
  img_dir_path = f'{SG_IMG_DIR}/val/{genre}'
  for filename in os.listdir(img_dir_path):
    img_path = f'{img_dir_path}/{filename}'
    im = cv2.imread(img_path)
    try:
      if im.shape not in img_shapes:
        img_shapes[im.shape] = 1
      else:
        img_shapes[im.shape] += 1
    except Exception as e:
      print(img_path)
      continue

img_shapes

{(450, 610, 3): 300}

In [ ]:
import cv2

img_shapes = {}
for idx, genre in enumerate(['Rock','Electronic','Pop']):
  img_dir_path = f'{SG_IMG_DIR}/val/{genre}'
  for filename in os.listdir(img_dir_path):
    img_path = f'{img_dir_path}/{filename}'
    im = cv2.imread(img_path)
    try:
      if im.shape not in img_shapes:
        img_shapes[im.shape] = 1
      else:
        img_shapes[im.shape] += 1
    except Exception as e:
      print(img_path)
      continue

img_shapes

{(450, 610, 3): 300}